In [1]:
import pandas as pd
import io
import numpy as np
import pymc3 as pm

from sklearn.preprocessing import StandardScaler


In [2]:
with open('german.data', 'r') as f:
    data_str = f.read()


In [3]:
# reading file 
data = io.StringIO(data_str)
df = pd.read_csv(data,sep=' ', header=None)
df.columns = ['status', 'months', 'credit', 'purpose', 'amount', 'savings', 'employment', 'rate', 'sex', 'other', 'residence',
            'property', 'age', 'plan', 'housing', 'nb_credits', 'job', 'liability', 'phone', 'foreign', 'target']
df.loc[df['target']==1, 'target'] =0
df.loc[df['target']==2, 'target'] =1

# replace categories by integers
std_scaler = StandardScaler()

for c in df.columns[:-1]:
    if df[c].dtype != np.int64:
        # convert to categorical
        df[c] = df[c].astype('category').cat.codes
    else:
        # normalize
        df[c] = (df[c] -df[c].mean())/df[c].std()

df.head()

,status,months,credit,purpose,amount,savings,employment,rate,sex,other,...,property,age,plan,housing,nb_credits,job,liability,phone,foreign,target
0,0,-1.235859,4,4,-0.744759,4,4,0.918018,2,0,...,0,2.765073,2,1,1.026565,2,-0.428075,1,0,0
1,1,2.247070,2,4,0.949342,0,2,-0.869748,1,0,...,0,-1.190808,2,1,-0.704573,2,-0.428075,0,0,1
2,3,-0.738298,4,7,-0.416354,0,3,-0.869748,2,0,...,0,1.182721,2,1,-0.704573,1,2.333701,0,0,0
3,0,1.749509,2,3,1.633430,0,3,-0.869748,2,2,...,1,0.831087,2,2,-0.704573,2,2.333701,0,0,0
4,0,0.256825,3,0,0.566380,0,2,0.024135,2,0,...,3,1.534354,2,2,1.026565,2,2.333701,0,0,1


In [5]:
df.iloc[:,:-1].shape

(1000, 20)

In [6]:
df.columns

Index(['status', 'months', 'credit', 'purpose', 'amount', 'savings',
       'employment', 'rate', 'sex', 'other', 'residence', 'property', 'age',
       'plan', 'housing', 'nb_credits', 'job', 'liability', 'phone', 'foreign',
       'target'],
      dtype='object')

Here $Y_i$ is a discrete RV that can take two values 0 and 1. Thus the Bernouilli probability model is the best candidate for the data. If we note $\pi_i$ the probability of accepting the credit for an individual $i$, 
    $$Y_i|\pi_i \sim Bern(\pi_i)$$
    and the expected value is:
    $$E(Y_i|\pi_i)=\pi$$

Finally we can define a linear model where $X_i \in \mathbb{R}^d$ describes the features of an individual $i$ with, $\alpha \in \mathbb{R}$, the intercept and $\beta \in \mathbb{R}^d$ the vector of coefficients:
$$g(\pi_i) = \alpha + \beta X_i$$
Now our goal is to write the Bernouilli mean $\pi_i$ as a linear function of $\alpha, \beta$. We can assume $\pi_i$ depends on the features $X_i$ through the logit function.  Hence :
$$\frac{\pi_i}{1 - \pi_i} = e^{\alpha + \beta X_i}$$ 
and
$$\pi_i = \frac{e^{\alpha + \beta X_i}}{1 + e^{\alpha + \beta X_i}}$$

To complete the Bayesian logistic regression model we need to specify priors on our regression parameters. Here we use zero-mean normal priors with variance $\sigma^2 = 100$. This is a weak prior because a large variance specifies little information about the parameter. 

We can now use an MCMC method to approximate the posterior.


In [74]:
# initialise using MLE 
from sklearn.linear_model import LogisticRegression
X, y = df.iloc[:,:-1], df.iloc[:,-1]
X, y = X.to_numpy(), y.to_numpy()

lr = LogisticRegression()
lr.fit(X,y)
alpha = lr.intercept_
betas = lr.coef_

coefs = np.empty((21))
coefs[0] = alpha
coefs[1:]=betas

# add column of ones to make computation ez 
X = np.hstack((np.ones((1000,1)), X))



In [91]:
# sample 
# weak prior and they all have the sample 
mu = 0
var = 100
def log_posterior_param(X, y, betas, mu, var):
    # likelihood x prior
    # we append to X a column of ones at the beginning
    prob = np.exp(betas@X.T) / (1 + np.exp(betas@X.T) )
    like = np.sum(y * np.log(prob) + (1-y) * np.log(1-prob))
    prior = np.sum(np.log(1/np.sqrt(2*np.pi*var)) - 0.5 * ((betas -mu)**2 / var))
    return like + prior 


def log_posterior(betas):
    print(glog_posterior_param(X, y, betas, mu, var)))
    return log_posterior_param(X, y, betas, mu, var)

def grad_logp(betas):
    print('making call')
    return grad(log_posterior(betas))




In [68]:
coefs.shape

(1, 21)

In [71]:
X.shape

(1000, 21)

In [12]:
import jax.numpy as jnp
from jax import grad 



In [92]:
from hmc import HamiltonianMonteCarlo
eps, L = 0.03, 20  # sampler
seed = 1
sampler = HamiltonianMonteCarlo(eps=eps, L=L, 
                                logp=log_posterior, grad_logp=grad_logp,
                                num_iterations=1000,
                                seed=seed)
sampler.run(coefs)

making call


TypeError: log_posterior() missing 4 required positional arguments: 'y', 'betas', 'mus', and 'sigmas'

In [18]:
sampler.run()

In [7]:
model = 'target ~ status + months + credit + purpose + amount + savings + employment + rate + sex \
    + other + residence + property + age + plan + housing + nb_credits + job + liability + phone \
        + foreign'

In [8]:
with pm.Model() as logistic_model:
    pm.glm.GLM.from_formula(formula=model, data=df, family=pm.glm.families.Binomial())

In [9]:
logistic_model.basic_RVs

[Intercept ~ Flat,
 status ~ Normal,
 months ~ Normal,
 credit ~ Normal,
 purpose ~ Normal,
 amount ~ Normal,
 savings ~ Normal,
 employment ~ Normal,
 rate ~ Normal,
 sex ~ Normal,
 other ~ Normal,
 residence ~ Normal,
 property ~ Normal,
 age ~ Normal,
 plan ~ Normal,
 housing ~ Normal,
 nb_credits ~ Normal,
 job ~ Normal,
 liability ~ Normal,
 phone ~ Normal,
 foreign ~ Normal,
 y ~ Binomial]

In [10]:
with logistic_model:
    trace = pm.sample(200)

/tmp/ipykernel_12459/558954564.py:2: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace = pm.sample(200)
Only 200 samples in chain.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [foreign, phone, liability, job, nb_credits, housing, plan, age, property, residence, other, sex, rate, employment, savings, amount, purpose, credit, months, status, Intercept]


In [ ]:
lower=-10**6
higher=10**6
with pm.Model() as model:
    # both alpha and beta are given weak zero-mean normal prior with variance =100
    alpha = pm.Normal('a', mu=0, sigma=np.sqrt(100))
    beta = pm.Normal('b',mu =0, sigma=np.sqrt(100), shape=(20))
    #beta_0=pm.Uniform('beta_0', lower=lower, upper= higher)
    #beta_age=pm.Uniform('beta_age', lower, higher)
    # define proba of belonging to class 1 
    p = pm.Deterministic('p', pm.math.sigmoid(alpha+beta@df.iloc[:,:-1].T))

with model:
    #fit the data 
    observed=pm.Bernoulli("target", p, observed=df['target'])
    start= pm.find_MAP()
    step= pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(2500, step=step, start=start)
    burned_trace=trace[15000:]
                      


/tmp/ipykernel_4805/211686632.py:19: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  trace=pm.sample(2500, step=step, start=start)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>Metropolis: [b]
>Metropolis: [a]
